In [ ]:
import pandas as pd

# Importing Train Dataset for Intial EDA
data = pd.read_csv("../Student Placement Dataset/train.csv")

C:\Users\SeanPC\AppData\Local\Temp\ipykernel_30104\4265757251.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
